In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
sales = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
shop = pd.read_csv('shops.csv')
item_cats = pd.read_csv("item_categories.csv")
test = pd.read_csv("test.csv")

In [4]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [7]:
sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')

In [8]:
sales.dtypes

date              datetime64[ns]
date_block_num             int64
shop_id                    int64
item_id                    int64
item_price               float64
item_cnt_day             float64
dtype: object

In [9]:
sales.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [10]:
sales.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')

In [11]:
sales.shape

(2935849, 6)

In [12]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [13]:
items.shape

(22170, 3)

In [14]:
items.describe()

,item_id,item_category_id
count,22170.00000,22170.000000
mean,11084.50000,46.290753
std,6400.07207,15.941486
min,0.00000,0.000000
25%,5542.25000,37.000000
50%,11084.50000,40.000000
75%,16626.75000,58.000000
max,22169.00000,83.000000


In [15]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [16]:
items.dtypes

item_name           object
item_id              int64
item_category_id     int64
dtype: object

In [17]:
items.isnull().sum()

item_name           0
item_id             0
item_category_id    0
dtype: int64

In [18]:
items.columns

Index(['item_name', 'item_id', 'item_category_id'], dtype='object')

In [19]:
shop.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [20]:
shop.isnull().sum()

shop_name    0
shop_id      0
dtype: int64

In [21]:
shop.columns

Index(['shop_name', 'shop_id'], dtype='object')

In [22]:
shop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [23]:
shop.describe()

,shop_id
count,60.000000
mean,29.500000
std,17.464249
min,0.000000
25%,14.750000
50%,29.500000
75%,44.250000
max,59.000000


In [24]:
item_cats.describe()

,item_category_id
count,84.000000
mean,41.500000
std,24.392622
min,0.000000
25%,20.750000
50%,41.500000
75%,62.250000
max,83.000000


In [25]:
item_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [26]:
item_cats.isnull().sum()

item_category_name    0
item_category_id      0
dtype: int64

In [27]:
item_cats.dtypes

item_category_name    object
item_category_id       int64
dtype: object

In [28]:
item_cats.columns

Index(['item_category_name', 'item_category_id'], dtype='object')

In [29]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [30]:
test.columns

Index(['ID', 'shop_id', 'item_id'], dtype='object')

In [31]:
test.dtypes

ID         int64
shop_id    int64
item_id    int64
dtype: object

In [32]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


In [33]:
test.isnull().sum()

ID         0
shop_id    0
item_id    0
dtype: int64

In [34]:
sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')
sales['month'] = sales['date'].dt.to_period('M')
sales = sales.groupby(['shop_id', 'item_id', 'month'], as_index=False)['item_cnt_day'].sum()
sales.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
sales['item_cnt_month'] = sales['item_cnt_month'].clip(0, 20)

In [36]:
sales['shop_id'] = sales['shop_id'].astype(np.int16)
sales['item_id'] = sales['item_id'].astype(np.int16)
sales['item_cnt_month'] = sales['item_cnt_month'].astype(np.float32)
sales['month'] = sales['month'].astype(str) 

In [37]:
def add_lag_feature(df, lag, col):
    tmp = df[['shop_id', 'item_id', 'month', col]].copy()
    tmp['month'] = pd.to_datetime(tmp['month'])
    tmp['month'] = (tmp['month'] + pd.DateOffset(months=lag)).dt.to_period('M').astype(str)
    tmp.rename(columns={col: f"{col}_lag_{lag}"}, inplace=True)
    return tmp

In [38]:
for lag in [1, 2, 3]:
    lag_feature = add_lag_feature(sales, lag, 'item_cnt_month')
    sales = pd.merge(sales, lag_feature, on=['shop_id', 'item_id', 'month'], how='left')
    del lag_feature 
    sales.fillna(0, inplace=True)

In [39]:
sales['month_dt'] = pd.to_datetime(sales['month'])
sales['month_num'] = sales['month_dt'].dt.month.astype(np.int8)
sales['year'] = sales['month_dt'].dt.year.astype(np.int16)
sales.drop(columns=['month_dt'], inplace=True)

In [40]:
X = sales.drop(columns=['item_cnt_month'])
y = sales['item_cnt_month']

In [41]:
X_train = X[X['month'] < '2015-10']
y_train = y[X['month'] < '2015-10']

In [42]:
X_valid = X[X['month'] == '2015-10']
y_valid = y[X['month'] == '2015-10']

In [43]:
X_train = X_train.drop(columns=['month'])
X_valid = X_valid.drop(columns=['month'])

In [44]:
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6
)

In [45]:
model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=True
)

[0]	validation_0-rmse:2.47940
[1]	validation_0-rmse:2.43116
[2]	validation_0-rmse:2.39059
[3]	validation_0-rmse:2.36021
[4]	validation_0-rmse:2.33492
[5]	validation_0-rmse:2.31233
[6]	validation_0-rmse:2.29510
[7]	validation_0-rmse:2.28152
[8]	validation_0-rmse:2.27080
[9]	validation_0-rmse:2.26247
[10]	validation_0-rmse:2.25472
[11]	validation_0-rmse:2.24814
[12]	validation_0-rmse:2.24400
[13]	validation_0-rmse:2.24055
[14]	validation_0-rmse:2.23673
[15]	validation_0-rmse:2.23512
[16]	validation_0-rmse:2.23317
[17]	validation_0-rmse:2.23225
[18]	validation_0-rmse:2.23040
[19]	validation_0-rmse:2.22888
[20]	validation_0-rmse:2.22797
[21]	validation_0-rmse:2.22606
[22]	validation_0-rmse:2.22533
[23]	validation_0-rmse:2.22478
[24]	validation_0-rmse:2.22345
[25]	validation_0-rmse:2.22191
[26]	validation_0-rmse:2.22104
[27]	validation_0-rmse:2.22072
[28]	validation_0-rmse:2.22023
[29]	validation_0-rmse:2.21991
[30]	validation_0-rmse:2.21895
[31]	validation_0-rmse:2.21885
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [46]:
preds = model.predict(X_valid)
preds = np.clip(preds, 0, 20)

In [47]:
rmse = np.sqrt(mean_squared_error(y_valid, preds))
mae = mean_absolute_error(y_valid, preds)

In [48]:
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

RMSE: 2.1415708359628867
MAE: 1.0635634660720825


In [49]:
import joblib
joblib.dump(model, 'model.joblib')
print("Model saved successfully!")

Model saved successfully!
